In [183]:
import numpy as np
import json
import pickle
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [184]:
train_data = []

with open('../data/domain1_train.json', 'r') as file:
    for line in file:
        data = json.loads(line)
        train_data.append(data)
len(train_data)

19500

In [155]:
#data cleaning
# texts = [data["text"] for data in train_data]

# # 设置所需的固定长度（假设为100）
# max_length = 100

# # 对文本进行padding
# padded_texts = pad_sequences(texts, maxlen=max_length, padding='post', truncating='post')

# # 将padding后的文本替换原来的文本
# for i in range(len(train_data)):
#     train_data[i]["text"] = padded_texts[i].tolist()
# len(train_data)

19500

In [185]:
X = np.array([entry["text"] for entry in train_data])
y = np.array([entry["label"] for entry in train_data])

<ipython-input-185-07b0c7f8f9f5>:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  X = np.array([entry["text"] for entry in train_data])


In [186]:
#split training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [187]:
# BOW n-grams
#vectorizer = HashingVectorizer(n_features=10000)
vectorizer = CountVectorizer(ngram_range=(1,2))

X_train_text = [' '.join(map(str, sample)) for sample in X_train]
X_test_text = [' '.join(map(str, sample)) for sample in X_test]

X_train_transformed = vectorizer.fit_transform(X_train_text)
X_test_transformed = vectorizer.transform(X_test_text)

In [188]:
logistic_model = LogisticRegression(penalty='l2',max_iter=10000)

### 插入部分

In [190]:
param_grid = {
    'C': [0.1, 1, 10],
    'multi_class': ['ovr', 'multinomial'],
    'class_weight': [None, 'balanced', {0: 1, 1: 2}] 
}
grid_search = GridSearchCV(logistic_model, param_grid, cv=10)

In [191]:
grid_search = GridSearchCV(logistic_model, param_grid, cv=10)

In [192]:
grid_search.fit(X_train_transformed, y_train)

GridSearchCV(cv=10, estimator=LogisticRegression(max_iter=10000),
             param_grid={'C': [0.1, 1, 10],
                         'class_weight': [None, 'balanced', {0: 1, 1: 2}],
                         'multi_class': ['ovr', 'multinomial']})

In [193]:
best_model = grid_search.best_estimator_
best_model.fit(X_train_transformed, y_train)

LogisticRegression(C=10, max_iter=10000, multi_class='ovr')

In [196]:
y_pred = best_model.predict(X_test_transformed)
accuracy = accuracy_score(y_test, y_pred)
print(f"Test Accuracy: {accuracy}")

Test Accuracy: 0.9302564102564103


### 插入结束

In [354]:
logistic_model.fit(X_train_transformed, y_train)

LogisticRegression()

In [355]:
y_pred = logistic_model.predict(X_test_transformed)

# output acc
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

Accuracy: 0.926923076923077


In [154]:
with open('../data/output_csv/logistic_output.csv', 'w') as output_file:
    output_file.write('id,class\n')  

    # read in dataset
    with open('../data/test_set.json', 'r') as file:
        for line in file:
            entry = json.loads(line)
            text = entry["text"]

            # word embedding
            X_test = vectorizer.transform([" ".join(map(str, text))])

            # use logistic model to predict
            prediction = best_model.predict(X_test)

            # output csv
            output_file.write(f"{entry['id']},{prediction[0]}\n")